In [4]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq

In [5]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

# Access the API key
api_key = os.getenv('API_KEY')
if not api_key:
    raise ValueError("API key not found. Please check your .env file.")
#print("API Key:", api_key)

In [6]:

llm = ChatGroq(model="llama3-8b-8192", api_key=api_key, temperature=0)

In [8]:
# Final code for Few shot prediction
# Define paths
data_path_1 = "./My_data"
test_file_path = os.path.join(data_path_1, "my_processed_data.csv")
data_path_2 = "./Processed"
train_file_path = os.path.join(data_path_2, "processedata_test.csv")

# Load the test and train data
test_data = pd.read_csv(test_file_path)
train_data = pd.read_csv(train_file_path)

# Function to get few-shot examples from the training data
def get_examples_from_csv(num_examples=3):
    examples = []
    for activity in train_data['Activity'].unique():
        df_activity = train_data[train_data['Activity'] == activity]
        for index, row in df_activity.head(num_examples).iterrows():
            example = f"""
{index + 1}. Features:
   - Mean_X: {row['Mean_X']}
   - StdDev_X: {row['StdDev_X']}
   - Min_X: {row['Min_X']}
   - Max_X: {row['Max_X']}
   - Mean_Y: {row['Mean_Y']}
   - StdDev_Y: {row['StdDev_Y']}
   - Min_Y: {row['Min_Y']}
   - Max_Y: {row['Max_Y']}
   - Mean_Z: {row['Mean_Z']}
   - StdDev_Z: {row['StdDev_Z']}
   - Min_Z: {row['Min_Z']}
   - Max_Z: {row['Max_Z']}
   Activity: {row['Activity']}
"""
            examples.append(example.strip())
    
    return "\n\n".join(examples)

def construct_prompt(features):
    examples = get_examples_from_csv()
    
    prompt = f"""
You are an expert in activity recognition using accelerometer data.
Based on the provided features and few-shot examples, predict the human activity performed.
**Give the predicted label in this format only: Label: <predicted label>**

Few-Shot Examples:
{examples.strip()}

Features:
- Mean_X: {features['Mean_X']}
- StdDev_X: {features['StdDev_X']}
- Min_X: {features['Min_X']}
- Max_X: {features['Max_X']}
- Mean_Y: {features['Mean_Y']}
- StdDev_Y: {features['StdDev_Y']}
- Min_Y: {features['Min_Y']}
- Max_Y: {features['Max_Y']}
- Mean_Z: {features['Mean_Z']}
- StdDev_Z: {features['StdDev_Z']}
- Min_Z: {features['Min_Z']}
- Max_Z: {features['Max_Z']}

Predict the activity as one of the following: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING.
**Remember to output only the label in the specified format.**
"""
    return prompt

def predict_activity(features):
    prompt = construct_prompt(features)
    response = llm.invoke(prompt)  
    predicted_activity = response.content.strip()  # Adjust based on the actual response format
    
    # print(f"Prompt:\n{prompt}")  # Debugging: Print prompt to verify it's correct
    # print(f"Response:\n{response.content}")  # Debugging: Print model response to check the output
    
    # Extract the predicted label from the model's response
    if predicted_activity.startswith("Label: "):
        predicted_activity = predicted_activity.replace("Label: ", "").strip()
    
    return predicted_activity

# Initialize variables for accuracy calculation
correct_predictions = 0
total_predictions = len(test_data)

# Create a list to store predictions and actual activities
results = []

# Process data in batches (e.g., 10 rows at a time)
batch_size = 10

for start in range(0, total_predictions, batch_size):
    end = min(start + batch_size, total_predictions)
    batch = test_data.iloc[start:end]
    for index, row in batch.iterrows():
        features = {
            "Mean_X": row['Mean_X'],
            "StdDev_X": row['StdDev_X'],
            "Min_X": row['Min_X'],
            "Max_X": row['Max_X'],
            "Mean_Y": row['Mean_Y'],
            "StdDev_Y": row['StdDev_Y'],
            "Min_Y": row['Min_Y'],
            "Max_Y": row['Max_Y'],
            "Mean_Z": row['Mean_Z'],
            "StdDev_Z": row['StdDev_Z'],
            "Min_Z": row['Min_Z'],
            "Max_Z": row['Max_Z']
        }
        actual_activity = row['Activity']
        predicted_activity = predict_activity(features)
        
        # Append results to the list
        results.append({
            'Index': index,
            'Features': features,
            'Actual Activity': actual_activity,
            'Predicted Activity': predicted_activity
        })
        
        # # Debug information
        # #print(f"Features: {features}")
        print(f"Actual Activity: {actual_activity}")
        print(f"Predicted Activity: {predicted_activity}")
        
        # Compare activities
        if predicted_activity.strip().upper() == actual_activity.strip().upper():
            correct_predictions += 1
        else:
            print(f"Mismatch: Predicted '{predicted_activity.strip().upper()}' vs Actual '{actual_activity.strip().upper()}'")

# Calculate accuracy
accuracy = (correct_predictions / total_predictions) * 100

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

# Print the accuracy
print(f"Accuracy of few-shot prediction: {accuracy:.2f}%")

# Save the results to a CSV file
#results_df.to_csv('activity_predictions_vs_actuals.csv', index=False)
#print("Results saved to activity_predictions_vs_actuals.csv")


Actual Activity: LAYING
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'LAYING'
Actual Activity: LAYING
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'LAYING'
Actual Activity: LAYING
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'LAYING'
Actual Activity: LAYING
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'LAYING'
Actual Activity: LAYING
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'LAYING'
Actual Activity: LAYING
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'LAYING'
Actual Activity: SITTING
Predicted Activity: SITTING
Actual Activity: SITTING
Predicted Activity: SITTING
Actual Activity: SITTING
Predicted Activity: SITTING
Actual Activity: SITTING
Predicted Activity: SITTING
Actual Activity: SITTING
Predicted Activity: SITTING
Actual Activity: SITTING
Predicted Activity: SITTING
Actual Activity: STANDING
Predicted Activity: SITTING
Mismatch: Predicted 'S

In [9]:
# Final code for Few shot prediction
# Define paths
data_path = "./My_data"
test_file_path = os.path.join(data_path, "my_processed_data_test.csv")
train_file_path = os.path.join(data_path, "my_processed_data_train.csv")

# Load the test and train data
test_data = pd.read_csv(test_file_path)
train_data = pd.read_csv(train_file_path)

# Function to get few-shot examples from the training data
def get_examples_from_csv(num_examples=3):
    examples = []
    for activity in train_data['Activity'].unique():
        df_activity = train_data[train_data['Activity'] == activity]
        for index, row in df_activity.head(num_examples).iterrows():
            example = f"""
{index + 1}. Features:
   - Mean_X: {row['Mean_X']}
   - StdDev_X: {row['StdDev_X']}
   - Min_X: {row['Min_X']}
   - Max_X: {row['Max_X']}
   - Mean_Y: {row['Mean_Y']}
   - StdDev_Y: {row['StdDev_Y']}
   - Min_Y: {row['Min_Y']}
   - Max_Y: {row['Max_Y']}
   - Mean_Z: {row['Mean_Z']}
   - StdDev_Z: {row['StdDev_Z']}
   - Min_Z: {row['Min_Z']}
   - Max_Z: {row['Max_Z']}
   Activity: {row['Activity']}
"""
            examples.append(example.strip())
    
    return "\n\n".join(examples)

def construct_prompt(features):
    examples = get_examples_from_csv()
    
    prompt = f"""
You are an expert in activity recognition using accelerometer data.
Based on the provided features and few-shot examples, predict the human activity performed.
**Give the predicted label in this format only: Label: <predicted label>**

Few-Shot Examples:
{examples.strip()}

Features:
- Mean_X: {features['Mean_X']}
- StdDev_X: {features['StdDev_X']}
- Min_X: {features['Min_X']}
- Max_X: {features['Max_X']}
- Mean_Y: {features['Mean_Y']}
- StdDev_Y: {features['StdDev_Y']}
- Min_Y: {features['Min_Y']}
- Max_Y: {features['Max_Y']}
- Mean_Z: {features['Mean_Z']}
- StdDev_Z: {features['StdDev_Z']}
- Min_Z: {features['Min_Z']}
- Max_Z: {features['Max_Z']}

Predict the activity as one of the following: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING.
**Remember to output only the label in the specified format.**
"""
    return prompt

def predict_activity(features):
    prompt = construct_prompt(features)
    response = llm.invoke(prompt)  
    predicted_activity = response.content.strip()  # Adjust based on the actual response format
    
    # print(f"Prompt:\n{prompt}")  # Debugging: Print prompt to verify it's correct
    # print(f"Response:\n{response.content}")  # Debugging: Print model response to check the output
    
    # Extract the predicted label from the model's response
    if predicted_activity.startswith("Label: "):
        predicted_activity = predicted_activity.replace("Label: ", "").strip()
    
    return predicted_activity

# Initialize variables for accuracy calculation
correct_predictions = 0
total_predictions = len(test_data)

# Create a list to store predictions and actual activities
results = []

# Process data in batches (e.g., 10 rows at a time)
batch_size = 10

for start in range(0, total_predictions, batch_size):
    end = min(start + batch_size, total_predictions)
    batch = test_data.iloc[start:end]
    for index, row in batch.iterrows():
        features = {
            "Mean_X": row['Mean_X'],
            "StdDev_X": row['StdDev_X'],
            "Min_X": row['Min_X'],
            "Max_X": row['Max_X'],
            "Mean_Y": row['Mean_Y'],
            "StdDev_Y": row['StdDev_Y'],
            "Min_Y": row['Min_Y'],
            "Max_Y": row['Max_Y'],
            "Mean_Z": row['Mean_Z'],
            "StdDev_Z": row['StdDev_Z'],
            "Min_Z": row['Min_Z'],
            "Max_Z": row['Max_Z']
        }
        actual_activity = row['Activity']
        predicted_activity = predict_activity(features)
        
        # Append results to the list
        results.append({
            'Index': index,
            'Features': features,
            'Actual Activity': actual_activity,
            'Predicted Activity': predicted_activity
        })
        
        # # Debug information
        # #print(f"Features: {features}")
        print(f"Actual Activity: {actual_activity}")
        print(f"Predicted Activity: {predicted_activity}")
        
        # Compare activities
        if predicted_activity.strip().upper() == actual_activity.strip().upper():
            correct_predictions += 1
        else:
            print(f"Mismatch: Predicted '{predicted_activity.strip().upper()}' vs Actual '{actual_activity.strip().upper()}'")

# Calculate accuracy
accuracy = (correct_predictions / total_predictions) * 100

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)

# Print the accuracy
print(f"Accuracy of few-shot prediction: {accuracy:.2f}%")

# Save the results to a CSV file
#results_df.to_csv('activity_predictions_vs_actuals.csv', index=False)
#print("Results saved to activity_predictions_vs_actuals.csv")


Actual Activity: WALKING_UPSTAIRS
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'WALKING_UPSTAIRS'
Actual Activity: WALKING_DOWNSTAIRS
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'WALKING_DOWNSTAIRS'
Actual Activity: STANDING
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'STANDING'
Actual Activity: WALKING_UPSTAIRS
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'WALKING_UPSTAIRS'
Actual Activity: SITTING
Predicted Activity: SITTING
Actual Activity: STANDING
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'STANDING'
Actual Activity: LAYING
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'LAYING'
Actual Activity: WALKING
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'WALKING'
Actual Activity: SITTING
Predicted Activity: SITTING
Actual Activity: WALKING_DOWNSTAIRS
Predicted Activity: SITTING
Mismatch: Predicted 'SITTING' vs Actual 'WALKING_